In [5]:
import os
import json
import ast

folder = "results/evaluation_small/no_replace_comma/gpt-3.5-turbo-16k"

result_files = os.listdir(folder)
result_files = [os.path.join(folder, file) for file in result_files if os.path.splitext(file)[1] == ".json"]
accuracies = []

for i, result_file in enumerate(result_files):

    task_name = result_file.split("/")[-1].split("_out")[0]
    true_task_path = os.path.join("data/evaluation_small/", task_name + ".json")

    ground_truth = None 

    with open(true_task_path, "r") as f:
        obj = json.loads(f.read())
        ground_truth = obj["test"][0]["output"]

    with open(result_file, "r") as f:
        results = json.loads(f.read())["output"]

        considered_results = []
        dimensions = []

        # for each result filter that all row dimensions are the same
        for result in results:
            try:
                result =  ast.literal_eval(result)
            except:
                continue
            d1 = len(result)
            if not isinstance(result[0], list): 
                continue

            d2 = len(result[0])
            invalid = False 

            for row in result: 
                
                if len(row) != d2: 
                    invalid = True 

            if not invalid: 
                considered_results.append(result)
                dimensions.append((d1, d2))


        # perform majority voting on the number of dimensions
        majority_dimension = max(dimensions,key=dimensions.count)
        # filter to lists with majority dimension
        considered_results = [result for result in considered_results if len(result) == majority_dimension[0] and len(result[0]) == majority_dimension[1]]

        final_output = considered_results[0]
        # majority voting on final output -> we perform character by character voting
        for i in range(len(final_output)):
            for j in range(len(final_output[0])):
                
                char_counter = []
                for elm_list in considered_results:
                    char_counter.append(elm_list[i][j])

                max_character =  max(char_counter,key=char_counter.count)
                final_output[i][j] = max_character

        correct = True

        for i in range(len(final_output)):
            for j in range(len(final_output[0])):
                try: 
                    if final_output[i][j] != ground_truth[i][j]:
                        correct = False
                except: 
                    correct = False

        if correct: 
            accuracies.append(1)
        else: 
            accuracies.append(0)

print("accuracy",  sum(accuracies), "/", len(accuracies))

accuracy 1 / 26
